In [3]:
import json

response = {
  "type": "message",
  "role": "assistant",
  "content": [
    {
      "type": "text",
      "text": "#%% [markdown]\n# 🔬 LAB: Model Monitoring with Scikit-Learn\n# การติดตามและเฝ้าระวังโมเดล Machine Learning\n\n#%% [markdown]\n## 📚 บทนำ (Introduction)\n# \n# ### Model Monitoring คืออะไร?\n# \n# **Model Monitoring** คือกระบวนการติดตามและเฝ้าระวังประสิทธิภาพของโมเดล Machine Learning \n# หลังจากที่ได้ deploy ไปใช้งานจริง (Production) เพื่อให้มั่นใจว่าโมเดลยังคงทำงานได้อย่างถูกต้อง\n# \n# ### ทำไมต้อง Monitor โมเดล?\n# \n# 1. **Data Drift** - ข้อมูลในโลกจริงเปลี่ยนแปลงตลอดเวลา\n# 2. **Concept Drift** - ความสัมพันธ์ระหว่าง features และ target เปลี่ยนไป\n# 3. **Model Degradation** - ประสิทธิภาพโมเดลลดลงเมื่อเวลาผ่านไป\n# 4. **Data Quality Issues** - ปัญหาคุณภาพข้อมูล เช่น missing values, outliers\n# \n# ### สิ่งที่จะเรียนรู้ใน LAB นี้\n# \n# - ✅ Data Quality Monitoring\n# - ✅ Model Performance Tracking\n# - ✅ Target Drift Detection\n# - ✅ Building Monitoring Dashboard\n\n#%% [markdown]\n# ---\n# ## 🛠️ Section 0: Environment Setup\n# ### การติดตั้งและเตรียมสภาพแวดล้อม\n\n#%%\n# ติดตั้ง libraries ที่จำเป็น (รันครั้งเดียว)\n# !pip install scikit-learn pandas numpy matplotlib seaborn scipy\n\n#%%\n# Import libraries ที่จำเป็น\nimport numpy as np\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom datetime import datetime, timedelta\nimport warnings\nwarnings.filterwarnings('ignore')\n\n# Scikit-learn imports\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.ensemble import RandomForestClassifier, RandomForestRegressor\nfrom sklearn.linear_model import LogisticRegression, LinearRegression\nfrom sklearn.preprocessing import StandardScaler, LabelEncoder\nfrom sklearn.metrics import (\n    accuracy_score, precision_score, recall_score, f1_score,\n    confusion_matrix, classification_report,\n    mean_absolute_error, mean_squared_error, r2_score\n)\n\n# Statistical tests\nfrom scipy import stats\nfrom scipy.stats import ks_2samp, chi2_contingency, wasserstein_distance\n\n# ตั้งค่า display\npd.set_option('display.max_columns', None)\npd.set_option('display.width', None)\nplt.style.use('seaborn-v0_8-whitegrid')\nplt.rcParams['figure.figsize'] = [12, 6]\nplt.rcParams['font.size'] = 10\n\nprint(\"✅ Import libraries สำเร็จ!\")\nprint(f\"📅 วันที่รัน: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\")\n\n#%% [markdown]\n# ---\n# ## 📊 Section 1: Data Quality Monitoring\n# ### การตรวจสอบคุณภาพข้อมูล\n# \n# **ทฤษฎี:**\n# \n# Data Quality Monitoring เป็นขั้นตอนแรกและสำคัญที่สุดในการ monitor โมเดล\n# เพราะ \"Garbage In, Garbage Out\" - ถ้าข้อมูลไม่ดี โมเดลก็ไม่สามารถทำนายได้ดี\n# \n# **สิ่งที่ต้องตรวจสอบ:**\n# 1. Missing Values - ค่าที่หายไป\n# 2. Duplicates - ข้อมูลซ้ำ\n# 3. Data Types - ประเภทข้อมูล\n# 4. Value Ranges - ช่วงค่าของข้อมูล\n# 5. Outliers - ค่าผิดปกติ\n\n#%% [markdown]\n# ### 1.1 สร้างข้อมูลตัวอย่าง (Synthetic Data)\n# \n# เราจะสร้างข้อมูลจำลองสำหรับปัญหา Credit Risk Classification\n# โดยมี 2 ชุด:\n# - **Reference Data** - ข้อมูลที่ใช้ train โมเดล (ข้อมูลในอดีต)\n# - **Current Data** - ข้อมูลใหม่ที่เข้ามา (ข้อมูลปัจจุบัน)\n\n#%%\ndef create_credit_data(n_samples=1000, seed=42, drift_level=0.0):\n    \"\"\"\n    สร้างข้อมูล Credit Risk จำลอง\n    \n    Parameters:\n    -----------\n    n_samples : int - จำนวนตัวอย่าง\n    seed : int - random seed\n    drift_level : float - ระดับ drift (0.0 = ไม่มี drift, 1.0 = drift มาก)\n    \n    Returns:\n    --------\n    DataFrame - ข้อมูล credit risk\n    \"\"\"\n    np.random.seed(seed)\n    \n    # สร้าง features\n    data = {\n        'customer_id': range(1, n_samples + 1),\n        'age': np.random.normal(35 + drift_level * 5, 10, n_samples).astype(int),\n        'income': np.random.normal(50000 + drift_level * 10000, 15000, n_samples),\n        'loan_amount': np.random.normal(20000 + drift_level * 5000, 8000, n_samples),\n        'credit_score': np.random.normal(650 - drift_level * 30, 80, n_samples).astype(int),\n        'employment_years': np.random.exponential(5 + drift_level, n_samples),\n        'num_credit_cards': np.random.poisson(3, n_samples),\n        'debt_to_income': np.random.uniform(0.1 + drift_level * 0.1, 0.6 + drift_level * 0.1, n_samples),\n        'previous_defaults': np.random.binomial(3, 0.1 + drift_level * 0.05, n_samples),\n        'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], \n                                       n_samples, p=[0.3, 0.4, 0.2, 0.1]),\n        'employment_type': np.random.choice(['Full-time', 'Part-time', 'Self-employed', 'Unemployed'],\n                                            n_samples, p=[0.6, 0.15, 0.2, 0.05])\n    }\n    \n    df = pd.DataFrame(data)\n    \n    # จำกัดค่าให้อยู่ในช่วงที่เหมาะสม\n    df['age'] = df['age'].clip(18, 80)\n    df['credit_score'] = df['credit_score'].clip(300, 850)\n    df['income'] = df['income'].clip(10000, 200000)\n    df['loan_amount'] = df['loan_amount'].clip(1000, 100000)\n    df['employment_years'] = df['employment_years'].clip(0, 40)\n    \n    # สร้าง target (default: 1 = ผิดนัดชำระ, 0 = ไม่ผิดนัด)\n    default_prob = (\n        0.1 +\n        0.02 * (df['debt_to_income'] - 0.3) * 10 +\n        0.01 * df['previous_defaults'] +\n        -0.001 * (df['credit_score'] - 600) +\n        -0.0001 * (df['income'] - 40000) +\n        drift_level * 0.1\n    )\n    default_prob = default_prob.clip(0.01, 0.99)\n    df['default'] = (np.random.random(n_samples) < default_prob).astype(int)\n    \n    return df\n\n# สร้าง Reference Data (ข้อมูลในอดีต - ไม่มี drift)\nprint(\"📊 กำลังสร้างข้อมูลตัวอย่าง...\")\nreference_data = create_credit_data(n_samples=2000, seed=42, drift_level=0.0)\nprint(f\"✅ Reference Data: {len(reference_data)} rows\")\n\n# สร้าง Current Data (ข้อมูลปัจจุบัน - มี drift เล็กน้อย)\ncurrent_data = create_credit_data(n_samples=1000, seed=123, drift_level=0.3)\nprint(f\"✅ Current Data: {len(current_data)} rows\")\n\n# แสดงตัวอย่างข้อมูล\nprint(\"\\n📋 ตัวอย่าง Reference Data:\")\nreference_data.head()\n\n#%%\n# แสดงข้อมูลเบื้องต้น\nprint(\"=\" * 60)\nprint(\"📊 REFERENCE DATA INFO\")\nprint(\"=\" * 60)\nprint(f\"Shape: {reference_data.shape}\")\nprint(f\"\\nData Types:\\n{reference_data.dtypes}\")\nprint(f\"\\nTarget Distribution:\\n{reference_data['default'].value_counts(normalize=True)}\")\n\n#%% [markdown]\n# ### 1.2 สร้าง Data Quality Report Class\n# \n# เราจะสร้าง class สำหรับตรวจสอบคุณภาพข้อมูลอย่างครบถ้วน\n\n#%%\nclass DataQualityMonitor:\n    \"\"\"\n    Class สำหรับตรวจสอบคุณภาพข้อมูล\n    \n    ใช้สำหรับ:\n    - ตรวจสอบ missing values\n    - ตรวจสอบ duplicates\n    - วิเคราะห์ data integrity\n    - สร้าง quality report\n    \"\"\"\n    \n    def __init__(self, data, data_name=\"Data\"):\n        \"\"\"\n        Parameters:\n        -----------\n        data : DataFrame - ข้อมูลที่ต้องการตรวจสอบ\n        data_name : str - ชื่อของชุดข้อมูล\n        \"\"\"\n        self.data = data.copy()\n        self.data_name = data_name\n        self.report = {}\n        \n    def check_missing_values(self):\n        \"\"\"ตรวจสอบ missing values\"\"\"\n        missing = self.data.isnull().sum()\n        missing_pct = (missing / len(self.data) * 100).round(2)\n        \n        missing_df = pd.DataFrame({\n            'column': missing.index,\n            'missing_count': missing.values,\n            'missing_percentage': missing_pct.values\n        })\n        missing_df = missing_df[missing_df['missing_count'] > 0].sort_values(\n            'missing_percentage', ascending=False\n        )\n        \n        self.report['missing_values'] = {\n            'total_missing': int(missing.sum()),\n            'columns_with_missing': len(missing_df),\n            'details': missing_df.to_dict('records')\n        }\n        \n        return missing_df\n    \n    def check_duplicates(self):\n        \"\"\"ตรวจสอบข้อมูลซ้ำ\"\"\"\n        # ตรวจสอบ duplicate rows ทั้งหมด\n        duplicate_rows = self.data.duplicated().sum()\n        \n        # ตรวจสอบ duplicate ตาม ID (ถ้ามี)\n        id_columns = [col for col in self.data.columns if 'id' in col.lower()]\n        duplicate_ids = {}\n        \n        for id_col in id_columns:\n            dup_count = self.data[id_col].duplicated().sum()\n            if dup_count > 0:\n                duplicate_ids[id_col] = int(dup_count)\n        \n        self.report['duplicates'] = {\n            'duplicate_rows': int(duplicate_rows),\n            'duplicate_percentage': round(duplicate_rows / len(self.data) * 100, 2),\n            'duplicate_ids': duplicate_ids\n        }\n        \n        return self.report['duplicates']\n    \n    def check_data_types(self):\n        \"\"\"ตรวจสอบประเภทข้อมูล\"\"\"\n        dtypes_df = pd.DataFrame({\n            'column': self.data.columns,\n            'dtype': self.data.dtypes.values,\n            'unique_values': [self.data[col].nunique() for col in self.data.columns],\n            'sample_values': [str(self.data[col].dropna().head(3).tolist()) for col in self.data.columns]\n        })\n        \n        # จำแนกประเภท columns\n        numeric_cols = self.data.select_dtypes(include=[np.number]).columns.tolist()\n        categorical_cols = self.data.select_dtypes(include=['object', 'category']).columns.tolist()\n        \n        self.report['data_types'] = {\n            'numeric_columns': numeric_cols,\n            'categorical_columns': categorical_cols,\n            'total_columns': len(self.data.columns)\n        }\n        \n        return dtypes_df\n    \n    def check_value_ranges(self):\n        \"\"\"ตรวจสอบช่วงค่าของข้อมูล numeric\"\"\"\n        numeric_cols = self.data.select_dtypes(include=[np.number]).columns\n        \n        range_stats = []\n        for col in numeric_cols:\n            stats_dict = {\n                'column': col,\n                'min': self.data[col].min(),\n                'max': self.data[col].max(),\n                'mean': round(self.data[col].mean(), 2),\n                'median': round(self.data[col].median(), 2),\n                'std': round(self.data[col].std(), 2),\n                'q1': round(self.data[col].quantile(0.25), 2),\n                'q3': round(self.data[col].quantile(0.75), 2)\n            }\n            range_stats.append(stats_dict)\n        \n        self.report['value_ranges'] = range_stats\n        return pd.DataFrame(range_stats)\n    \n    def detect_outliers(self, method='iqr', threshold=1.5):\n        \"\"\"\n        ตรวจหา outliers ด้วยวิธี IQR\n        \n        Parameters:\n        -----------\n        method : str - วิธีการตรวจหา ('iqr' หรือ 'zscore')\n        threshold : float - ค่า threshold (1.5 สำหรับ IQR, 3 สำหรับ z-score)\n        \"\"\"\n        numeric_cols = self.data.select_dtypes(include=[np.number]).columns\n        outlier_info = []\n        \n        for col in numeric_cols:\n            if method == 'iqr':\n                Q1 = self.data[col].quantile(0.25)\n                Q3 = self.data[col].quantile(0.75)\n                IQR = Q3 - Q1\n                lower_bound = Q1 - threshold * IQR\n                upper_bound = Q3 + threshold * IQR\n                outliers = self.data[(self.data[col] < lower_bound) | (self.data[col] > upper_bound)]\n            else:  # z-score\n                z_scores = np.abs(stats.zscore(self.data[col].dropna()))\n                outliers = self.data[z_scores > threshold]\n            \n            outlier_count = len(outliers)\n            outlier_pct = round(outlier_count / len(self.data) * 100, 2)\n            \n            outlier_info.append({\n                'column': col,\n                'outlier_count': outlier_count,\n                'outlier_percentage': outlier_pct,\n                'method': method,\n                'threshold': threshold\n            })\n        \n        self.report['outliers'] = outlier_info\n        return pd.DataFrame(outlier_info)\n    \n    def generate_full_report(self):\n        \"\"\"สร้าง report ฉบับเต็ม\"\"\"\n        print(\"=\" * 70)\n        print(f\"📊 DATA QUALITY REPORT: {self.data_name}\")\n        print(\"=\" * 70)\n        print(f\"📅 Generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\")\n        print(f\"📁 Total Records: {len(self.data):,}\")\n        print(f\"📋 Total Columns: {len(self.data.columns)}\")\n        print()\n        \n        # 1. Missing Values\n        print(\"-\" * 50)\n        print(\"1️⃣ MISSING VALUES CHECK\")\n        print(\"-\" * 50)\n        missing_df = self.check_missing_values()\n        if len(missing_df) > 0:\n            print(f\"⚠️ พบ {self.report['missing_values']['total_missing']} missing values\")\n            print(f\"   ใน {self.report['missing_values']['columns_with_missing']} columns\")\n            print(missing_df.to_string(index=False))\n        else:\n            print(\"✅ ไม่พบ missing values\")\n        print()\n        \n        # 2. Duplicates\n        print(\"-\" * 50)\n        print(\"2️⃣ DUPLICATES CHECK\")\n        print(\"-\" * 50)\n        dup_report = self.check_duplicates()\n        if dup_report['duplicate_rows'] > 0:\n            print(f\"⚠️ พบ {dup_report['duplicate_rows']} duplicate rows ({dup_report['duplicate_percentage']}%)\")\n        else:\n            print(\"✅ ไม่พบ duplicate rows\")\n        \n        if dup_report['duplicate_ids']:\n            print(f\"⚠️ Duplicate IDs: {dup_report['duplicate_ids']}\")\n        print()\n        \n        # 3. Data Types\n        print(\"-\" * 50)\n        print(\"3️⃣ DATA TYPES SUMMARY\")\n        print(\"-\" * 50)\n        dtypes_df = self.check_data_types()\n        print(f\"📊 Numeric Columns: {len(self.report['data_types']['numeric_columns'])}\")\n        print(f\"📝 Categorical Columns: {len(self.report['data_types']['categorical_columns'])}\")\n        print()\n        \n        # 4. Value Ranges\n        print(\"-\" * 50)\n        print(\"4️⃣ VALUE RANGES (Numeric Columns)\")\n        print(\"-\" * 50)\n        ranges_df = self.check_value_ranges()\n        print(ranges_df.to_string(index=False))\n        print()\n        \n        # 5. Outliers\n        print(\"-\" * 50)\n        print(\"5️⃣ OUTLIERS DETECTION (IQR Method)\")\n        print(\"-\" * 50)\n        outliers_df = self.detect_outliers()\n        outliers_with_issues = outliers_df[outliers_df['outlier_count'] > 0]\n        if len(outliers_with_issues) > 0:\n            print(f\"⚠️ พบ outliers ใน {len(outliers_with_issues)} columns:\")\n            print(outliers_with_issues.to_string(index=False))\n        else:\n            print(\"✅ ไม่พบ outliers ที่ผิดปกติ\")\n        \n        print()\n        print(\"=\" * 70)\n        print(\"📊 END OF DATA QUALITY REPORT\")\n        print(\"=\" * 70)\n        \n        return self.report\n\n#%%\n# ทดสอบ Data Quality Monitor กับ Reference Data\nprint(\"🔍 ตรวจสอบคุณภาพ Reference Data\")\nprint()\n\nref_monitor = DataQualityMonitor(reference_data, \"Reference Data\")\nref_report = ref_monitor.generate_full_report()\n\n#%%\n# ทดสอบกับ Current Data\nprint(\"\\n🔍 ตรวจสอบคุณภาพ Current Data\")\nprint()\n\ncurr_monitor = DataQualityMonitor(current_data, \"Current Data\")\ncurr_report = curr_monitor.generate_full_report()\n\n#%% [markdown]\n# ### 1.3 จำลองข้อมูลที่มีปัญหาคุณภาพ\n# \n# เพื่อทดสอบระบบ monitoring เราจะสร้างข้อมูลที่มีปัญหาคุณภาพ\n\n#%%\ndef introduce_data_quality_issues(data, missing_rate=0.05, duplicate_rate=0.02):\n    \"\"\"\n    เพิ่มปัญหาคุณภาพข้อมูลเพื่อทดสอบระบบ\n    \n    Parameters:\n    -----------\n    data : DataFrame - ข้อมูลต้นฉบับ\n    missing_rate : float - อัตราส่วน missing values\n    duplicate_rate : float - อัตราส่วน duplicates\n    \"\"\"\n    df = data.copy()\n    n_rows = len(df)\n    \n    # เพิ่ม missing values\n    missing_cols = ['income', 'credit_score', 'employment_years']\n    for col in missing_cols:\n        missing_idx = np.random.choice(n_rows, int(n_rows * missing_rate), replace=False)\n        df.loc[missing_idx, col] = np.nan\n    \n    # เพิ่ม duplicates\n    n_duplicates = int(n_rows * duplicate_rate)\n    duplicate_rows = df.sample(n=n_duplicates, random_state=42)\n    df = pd.concat([df, duplicate_rows], ignore_index=True)\n    \n    # เพิ่ม outliers\n    outlier_idx = np.random.choice(len(df), 20, replace=False)\n    df.loc[outlier_idx, 'income'] = df.loc[outlier_idx, 'income'] * 10  # รายได้สูงผิดปกติ\n    \n    return df\n\n# สร้างข้อมูลที่มีปัญหา\nproblematic_data = introduce_data_quality_issues(current_data.copy())\nprint(f\"📊 ข้อมูลที่มีปัญหา: {len(problematic_data)} rows\")\n\n# ตรวจสอบ\nprint(\"\\n🔍 ตรวจสอบข้อมูลที่มีปัญหา:\")\nproblem_monitor = DataQualityMonitor(problematic_data, \"Problematic Data\")\nproblem_report = problem_monitor.generate_full_report()\n\n#%% [markdown]\n# ### 1.4 Data Quality Alert System\n# \n# ระบบแจ้งเตือนเมื่อคุณภาพข้อมูลต่ำกว่าเกณฑ์ที่กำหนด\n\n#%%\nclass DataQualityAlert:\n    \"\"\"\n    ระบบแจ้งเตือนคุณภาพข้อมูล\n    \n    ตั้งค่า threshold และแจ้งเตือนเมื่อเกินค่าที่กำหนด\n    \"\"\"\n    \n    def __init__(self):\n        # Default thresholds\n        self.thresholds = {\n            'missing_rate': 0.05,      # ไม่เกิน 5% missing\n            'duplicate_rate': 0.01,    # ไม่เกิน 1% duplicates\n            'outlier_rate': 0.05       # ไม่เกิน 5% outliers\n        }\n        self.alerts = []\n    \n    def set_threshold(self, metric, value):\n        \"\"\"ตั้งค่า threshold\"\"\"\n        if metric in self.thresholds:\n            self.thresholds[metric] = value\n            print(f\"✅ ตั้งค่า {metric} threshold = {value}\")\n    \n    def check_alerts(self, data, data_name=\"Data\"):\n        \"\"\"ตรวจสอบและสร้าง alerts\"\"\"\n        self.alerts = []\n        n_rows = len(data)\n        \n        # Check missing values\n        missing_rate = data.isnull().sum().sum() / (n_rows * len(data.columns))\n        if missing_rate > self.thresholds['missing_rate']:\n            self.alerts.append({\n                'type': 'CRITICAL' if missing_rate > 0.1 else 'WARNING',\n                'metric': 'Missing Values',\n                'current_value': f\"{missing_rate:.2%}\",\n                'threshold': f\"{self.thresholds['missing_rate']:.2%}\",\n                'message': f\"Missing value rate ({missing_rate:.2%}) เกิน threshold\"\n            })\n        \n        # Check duplicates\n        duplicate_rate = data.duplicated().sum() / n_rows\n        if duplicate_rate > self.thresholds['duplicate_rate']:\n            self.alerts.append({\n                'type': 'WARNING',\n                'metric': 'Duplicates',\n                'current_value': f\"{duplicate_rate:.2%}\",\n                'threshold': f\"{self.thresholds['duplicate_rate']:.2%}\",\n                'message': f\"Duplicate rate ({duplicate_rate:.2%}) เกิน threshold\"\n            })\n        \n        # Check outliers (ใช้ IQR method)\n        numeric_cols = data.select_dtypes(include=[np.number]).columns\n        total_outliers = 0\n        for col in numeric_cols:\n            Q1 = data[col].quantile(0.25)\n            Q3 = data[col].quantile(0.75)\n            IQR = Q3 - Q1\n            outliers = data[(data[col] < Q1 - 1.5*IQR) | (data[col] > Q3 + 1.5*IQR)]\n            total_outliers += len(outliers)\n        \n        outlier_rate = total_outliers / (n_rows * len(numeric_cols))\n        if outlier_rate > self.thresholds['outlier_rate']:\n            self.alerts.append({\n                'type': 'WARNING',\n                'metric': 'Outliers',\n                'current_value': f\"{outlier_rate:.2%}\",\n                'threshold': f\"{self.thresholds['outlier_rate']:.2%}\",\n                'message': f\"Outlier rate ({outlier_rate:.2%}) เกิน threshold\"\n            })\n        \n        # แสดงผล alerts\n        self._display_alerts(data_name)\n        \n        return self.alerts\n    \n    def _display_alerts(self, data_name):\n        \"\"\"แสดงผล alerts\"\"\"\n        print(\"=\" * 70)\n        print(f\"🚨 DATA QUALITY ALERTS: {data_name}\")\n        print(\"=\" * 70)\n        \n        if not self.alerts:\n            print(\"✅ ไม่มี alerts - คุณภาพข้อมูลอยู่ในเกณฑ์ที่กำหนด\")\n        else:\n            print(f\"⚠️ พบ {len(self.alerts)} alerts:\")\n            print()\n            for i, alert in enumerate(self.alerts, 1):\n                icon = \"🔴\" if alert['type'] == 'CRITICAL' else \"🟡\"\n                print(f\"{icon} Alert #{i}: [{alert['type']}] {alert['metric']}\")\n                print(f\"   Current: {alert['current_value']} | Threshold: {alert['threshold']}\")\n                print(f\"   Message: {alert['message']}\")\n                print()\n        \n        print(\"=\" * 70)\n\n#%%\n# ทดสอบ Alert System\nalert_system = DataQualityAlert()\n\n# ตรวจสอบข้อมูลปกติ\nprint(\"🔍 ตรวจสอบ Reference Data:\")\nalerts_ref = alert_system.check_alerts(reference_data, \"Reference Data\")\n\nprint(\"\\n\")\n\n# ตรวจสอบข้อมูลที่มีปัญหา\nprint(\"🔍 ตรวจสอบ Problematic Data:\")\nalerts_problem = alert_system.check_alerts(problematic_data, \"Problematic Data\")\n\n#%% [markdown]\n# ### 1.5 Visualization: Data Quality Dashboard\n\n#%%\ndef plot_data_quality_summary(data, title=\"Data Quality Summary\"):\n    \"\"\"\n    สร้าง visualization สรุปคุณภาพข้อมูล\n    \"\"\"\n    fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n    fig.suptitle(f'📊 {title}', fontsize=14, fontweight='bold')\n    \n    # 1. Missing Values by Column\n    ax1 = axes[0, 0]\n    missing = data.isnull().sum()\n    missing = missing[missing > 0].sort_values(ascending=True)\n    if len(missing) > 0:\n        colors = ['red' if v > len(data)*0.05 else 'orange' for v in missing.values]\n        missing.plot(kind='barh', ax=ax1, color=colors)\n        ax1.set_title('Missing Values by Column')\n        ax1.set_xlabel('Count')\n    else:\n        ax1.text(0.5, 0.5, '✅ No Missing Values', ha='center', va='center', fontsize=12)\n        ax1.set_title('Missing Values by Column')\n    \n    # 2. Data Types Distribution\n    ax2 = axes[0, 1]\n    dtype_counts = data.dtypes.astype(str).value_counts()\n    colors = plt.cm.Set3(range(len(dtype_counts)))\n    dtype_counts.plot(kind='pie', ax=ax2, autopct='%1.1f%%', colors=colors)\n    ax2.set_title('Data Types Distribution')\n    ax2.set_ylabel('')\n    \n    # 3. Numeric Columns Distribution (Box plots)\n    ax3 = axes[1, 0]\n    numeric_cols = data.select_dtypes(include=[np.number]).columns[:6]  # แสดงแค่ 6 columns\n    if len(numeric_cols) > 0:\n        # Normalize data for comparison\n        normalized = (data[numeric_cols] - data[numeric_cols].mean()) / data[numeric_cols].std()\n        normalized.boxplot(ax=ax3)\n        ax3.set_title('Numeric Columns Distribution (Normalized)')\n        ax3.tick_params(axis='x', rotation=45)\n    \n    # 4. Quality Score Gauge\n    ax4 = axes[1, 1]\n    \n    # คำนวณ quality score\n    missing_score = max(0, 100 - (data.isnull().sum().sum() / (len(data) * len(data.columns)) * 100 * 10))\n    duplicate_score = max(0, 100 - (data.duplicated().sum() / len(data) * 100 * 10))\n    completeness_score = (1 - data.isnull().any(axis=1).sum() / len(data)) * 100\n    \n    overall_score = (missing_score + duplicate_score + completeness_score) / 3\n    \n    # สร้าง gauge chart ด้วย pie chart\n    sizes = [overall_score, 100 - overall_score]\n    colors_gauge = ['green' if overall_score >= 80 else 'orange' if overall_score >= 60 else 'red', 'lightgray']\n    ax4.pie(sizes, colors=colors_gauge, startangle=90, counterclock=False)\n    \n    # วาดวงกลมตรงกลาง\n    centre_circle = plt.Circle((0, 0), 0.70, fc='white')\n    ax4.add_patch(centre_circle)\n    ax4.text(0, 0, f'{overall_score:.1f}%', ha='center', va='center', fontsize=20, fontweight='bold')\n    ax4.text(0, -0.2, 'Quality Score', ha='center', va='center', fontsize=10)\n    ax4.set_title('Overall Data Quality Score')\n    \n    plt.tight_layout()\n    plt.show()\n    \n    return {\n        'missing_score': missing_score,\n        'duplicate_score': duplicate_score,\n        'completeness_score': completeness_score,\n        'overall_score': overall_score\n    }\n\n#%%\n# แสดง visualization สำหรับ Reference Data\nprint(\"📊 Reference Data Quality Summary:\")\nref_scores = plot_data_quality_summary(reference_data, \"Reference Data Quality Summary\")\n\n#%%\n# แสดง visualization สำหรับ Problematic Data\nprint(\"📊 Problematic Data Quality Summary:\")\nproblem_scores = plot_data_quality_summary(problematic_data, \"Problematic Data Quality Summary\")\n\n#%% [markdown]\n# ---\n# ## 📈 Section 2: Model Performance Tracking\n# ### การติดตามประสิทธิภาพโมเดล\n# \n# **ทฤษฎี:**\n# \n# Model Performance Tracking คือการติดตามว่าโมเดลทำนายได้ดีแค่ไหนเมื่อเวลาผ่านไป\n# \n# **Classification Metrics:**\n# - **Accuracy** - ความถูกต้องโดยรวม\n# - **Precision** - ความแม่นยำ (จากที่ทำนาย positive มีกี่ % ที่ถูก)\n# - **Recall** - ความครอบคลุม (จาก positive จริง ทำนายถูกกี่ %)\n# - **F1-Score** - ค่าเฉลี่ยฮาร์โมนิกของ Precision และ Recall\n# \n# **Regression Metrics:**\n# - **MAE (Mean Absolute Error)** - ค่าเฉลี่ยความคลาดเคลื่อนสัมบูรณ์\n# - **RMSE (Root Mean Squared Error)** - รากของค่าเฉลี่ยความคลาดเคลื่อนกำลังสอง\n# - **R² (R-squared)** - สัดส่วนความแปรปรวนที่อธิบายได้\n\n#%% [markdown]\n# ### 2.1 เตรียมข้อมูลและ Train โมเดล\n\n#%%\ndef prepare_features(data, target_col='default'):\n    \"\"\"\n    เตรียม features สำหรับ modeling\n    \n    Parameters:\n    -----------\n    data : DataFrame - ข้อมูลดิบ\n    target_col : str - ชื่อ column ที่เป็น target\n    \n    Returns:\n    --------\n    X : DataFrame - features\n    y : Series - target\n    \"\"\"\n    df = data.copy()\n    \n    # ลบ columns ที่ไม่ใช้\n    drop_cols = ['customer_id']\n    df = df.drop(columns=[col for col in drop_cols if col in df.columns])\n    \n    # แยก X และ y\n    y = df[target_col]\n    X = df.drop(columns=[target_col])\n    \n    # Encode categorical columns\n    categorical_cols = X.select_dtypes(include=['object']).columns\n    for col in categorical_cols:\n        le = LabelEncoder()\n        X[col] = le.fit_transform(X[col].astype(str))\n    \n    # Handle missing values\n    X = X.fillna(X.median())\n    \n    return X, y\n\n#%%\n# เตรียมข้อมูล Reference\nX_ref, y_ref = prepare_features(reference_data)\n\n# แบ่งข้อมูลสำหรับ train และ test\nX_train, X_test, y_train, y_test = train_test_split(\n    X_ref, y_ref, test_size=0.3, random_state=42, stratify=y_ref\n)\n\nprint(f\"📊 Training set: {len(X_train)} samples\")\nprint(f\"📊 Test set: {len(X_test)} samples\")\nprint(f\"\\n📋 Features: {list(X_train.columns)}\")\nprint(f\"\\n📊 Target distribution (train):\")\nprint(y_train.value_counts(normalize=True))\n\n#%%\n# Train โมเดล\nprint(\"🔧 กำลัง train โมเดล Random Forest...\")\n\n# Standardize features\nscaler = StandardScaler()\nX_train_scaled = scaler.fit_transform(X_train)\nX_test_scaled = scaler.transform(X_test)\n\n# Train Random Forest\nrf_model = RandomForestClassifier(\n    n_estimators=100,\n    max_depth=10,\n    random_state=42,\n    n_jobs=-1\n)\nrf_model.fit(X_train_scaled, y_train)\n\nprint(\"✅ Train โมเดลเสร็จสิ้น!\")\n\n# ทำนาย\ny_pred_train = rf_model.predict(X_train_scaled)\ny_pred_test = rf_model.predict(X_test_scaled)\ny_prob_test = rf_model.predict_proba(X_test_scaled)[:, 1]\n\nprint(f\"\\n📊 Training Accuracy: {accuracy_score(y_train, y_pred_train):.4f}\")\nprint(f\"📊 Test Accuracy: {accuracy_score(y_test, y_pred_test):.4f}\")\n\n#%% [markdown]\n# ### 2.2 สร้าง Model Performance Monitor Class\n\n#%%\nclass ModelPerformanceMonitor:\n    \"\"\"\n    Class สำหรับติดตามประสิทธิภาพโมเดล\n    \n    รองรับทั้ง Classification และ Regression\n    \"\"\"\n    \n    def __init__(self, model, model_name=\"Model\"):\n        \"\"\"\n        Parameters:\n        -----------\n        model : sklearn model - โมเดลที่ train แล้ว\n        model_name : str - ชื่อโมเดล\n        \"\"\"\n        self.model = model\n        self.model_name = model_name\n        self.performance_history = []\n        self.baseline_metrics = None\n        \n    def calculate_classification_metrics(self, y_true, y_pred, y_prob=None):\n        \"\"\"คำนวณ metrics สำหรับ classification\"\"\"\n        metrics = {\n            'accuracy': accuracy_score(y_true, y_pred),\n            'precision': precision_score(y_true, y_pred, average='weighted', zero_division=0),\n            'recall': recall_score(y_true, y_pred, average='weighted', zero_division=0),\n            'f1': f1_score(y_true, y_pred, average='weighted', zero_division=0),\n        }\n        \n        # Confusion matrix values\n        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()\n        metrics['true_positive'] = int(tp)\n        metrics['true_negative'] = int(tn)\n        metrics['false_positive'] = int(fp)\n        metrics['false_negative'] = int(fn)\n        \n        # Specificity\n        metrics['specificity'] = tn / (tn + fp) if (tn + fp) > 0 else 0\n        \n        return metrics\n    \n    def calculate_regression_metrics(self, y_true, y_pred):\n        \"\"\"คำนวณ metrics สำหรับ regression\"\"\"\n        metrics = {\n            'mae': mean_absolute_error(y_true, y_pred),\n            'mse': mean_squared_error(y_true, y_pred),\n            'rmse': np.sqrt(mean_squared_error(y_true, y_pred)),\n            'r2': r2_score(y_true, y_pred),\n            'mape': np.mean(np.abs((y_true - y_pred) / (y_true + 1e-10))) * 100\n        }\n        return metrics\n    \n    def set_baseline(self, y_true, y_pred, y_prob=None, task='classification'):\n        \"\"\"ตั้งค่า baseline metrics\"\"\"\n        if task == 'classification':\n            self.baseline_metrics = self.calculate_classification_metrics(y_true, y_pred, y_prob)\n        else:\n            self.baseline_metrics = self.calculate_regression_metrics(y_true, y_pred)\n        \n        self.baseline_metrics['timestamp'] = datetime.now()\n        self.baseline_metrics['task'] = task\n        \n        print(\"✅ ตั้งค่า Baseline Metrics เรียบร้อย:\")\n        self._print_metrics(self.baseline_metrics)\n        \n        return self.baseline_metrics\n    \n    def evaluate(self, X, y_true, data_name=\"Current\", task='classification'):\n        \"\"\"\n        ประเมินประสิทธิภาพโมเดลกับข้อมูลใหม่\n        \"\"\"\n        # ทำนาย\n        y_pred = self.model.predict(X)\n        y_prob = None\n        if task == 'classification' and hasattr(self.model, 'predict_proba'):\n            y_prob = self.model.predict_proba(X)[:, 1]\n        \n        # คำนวณ metrics\n        if task == 'classification':\n            current_metrics = self.calculate_classification_metrics(y_true, y_pred, y_prob)\n        else:\n            current_metrics = self.calculate_regression_metrics(y_true, y_pred)\n        \n        current_metrics['timestamp'] = datetime.now()\n        current_metrics['data_name'] = data_name\n        current_metrics['n_samples'] = len(y_true)\n        \n        # เก็บประวัติ\n        self.performance_history.append(current_metrics)\n        \n        # เปรียบเทียบกับ baseline\n        comparison = self._compare_with_baseline(current_metrics, task)\n        \n        # แสดงผล\n        self._display_evaluation(current_metrics, comparison, data_name, task)\n        \n        return current_metrics, comparison\n    \n    def _compare_with_baseline(self, current_metrics, task):\n        \"\"\"เปรียบเทียบกับ baseline\"\"\"\n        if self.baseline_metrics is None:\n            return None\n        \n        comparison = {}\n        if task == 'classification':\n            key_metrics = ['accuracy', 'precision', 'recall', 'f1']\n        else:\n            key_metrics = ['mae', 'rmse', 'r2']\n        \n        for metric in key_metrics:\n            baseline_val = self.baseline_metrics.get(metric, 0)\n            current_val = current_metrics.get(metric, 0)\n            \n            if task == 'classification':\n                # สำหรับ classification ค่าสูงกว่าดีกว่า\n                change = current_val - baseline_val\n                change_pct = (change / baseline_val * 100) if baseline_val != 0 else 0\n            else:\n                # สำหรับ regression (MAE, RMSE) ค่าต่ำกว่าดีกว่า (ยกเว้น R2)\n                if metric == 'r2':\n                    change = current_val - baseline_val\n                    change_pct = (change / baseline_val * 100) if baseline_val != 0 else 0\n                else:\n                    change = baseline_val - current_val\n                    change_pct = (change / baseline_val * 100) if baseline_val != 0 else 0\n            \n            comparison[metric] = {\n                'baseline': baseline_val,\n                'current': current_val,\n                'change': change,\n                'change_pct': change_pct,\n                'status': 'improved' if change > 0 else 'degraded' if change < 0 else 'stable'\n            }\n        \n        return comparison\n    \n    def _print_metrics(self, metrics):\n        \"\"\"แสดง metrics\"\"\"\n        for key, value in metrics.items():\n            if key not in ['timestamp', 'task', 'data_name', 'n_samples']:\n                if isinstance(value, float):\n                    print(f\"   {key}: {value:.4f}\")\n                else:\n                    print(f\"   {key}: {value}\")\n    \n    def _display_evaluation(self, metrics, comparison, data_name, task):\n        \"\"\"แสดงผลการประเมิน\"\"\"\n        print(\"=\" * 70)\n        print(f\"📊 MODEL PERFORMANCE EVALUATION: {data_name}\")\n        print(f\"   Model: {self.model_name}\")\n        print(f\"   Samples: {metrics['n_samples']:,}\")\n        print(f\"   Time: {metrics['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}\")\n        print(\"=\" * 70)\n        \n        if task == 'classification':\n            print(\"\\n📈 Classification Metrics:\")\n            print(\"-\" * 40)\n            print(f\"   Accuracy:  {metrics['accuracy']:.4f}\")\n            print(f\"   Precision: {metrics['precision']:.4f}\")\n            print(f\"   Recall:    {metrics['recall']:.4f}\")\n            print(f\"   F1-Score:  {metrics['f1']:.4f}\")\n            print(f\"\\n📊 Confusion Matrix:\")\n            print(f\"   TP: {metrics['true_positive']} | FP: {metrics['false_positive']}\")\n            print(f\"   FN: {metrics['false_negative']} | TN: {metrics['true_negative']}\")\n        else:\n            print(\"\\n📈 Regression Metrics:\")\n            print(\"-\" * 40)\n            print(f\"   MAE:  {metrics['mae']:.4f}\")\n            print(f\"   RMSE: {metrics['rmse']:.4f}\")\n            print(f\"   R²:   {metrics['r2']:.4f}\")\n        \n        if comparison:\n            print(\"\\n📊 Comparison with Baseline:\")\n            print(\"-\" * 40)\n            for metric, values in comparison.items():\n                icon = \"📈\" if values['status'] == 'improved' else \"📉\" if values['status'] == 'degraded' else \"➡️\"\n                print(f\"   {icon} {metric}: {values['baseline']:.4f} → {values['current']:.4f} ({values['change_pct']:+.2f}%)\")\n        \n        print(\"=\" * 70)\n\n#%%\n# สร้าง Performance Monitor\nperf_monitor = ModelPerformanceMonitor(rf_model, \"Random Forest Credit Risk\")\n\n# ตั้งค่า Baseline ด้วย Test Data\nprint(\"📊 ตั้งค่า Baseline Metrics:\")\nbaseline = perf_monitor.set_baseline(y_test, y_pred_test, y_prob_test, task='classification')\n\n#%%\n# ประเมินกับ Current Data (ข้อมูลใหม่ที่มี drift)\nX_curr, y_curr = prepare_features(current_data)\nX_curr_scaled = scaler.transform(X_curr)\n\nprint(\"\\n📊 ประเมินกับ Current Data (มี Drift):\")\ncurr_metrics, comparison = perf_monitor.evaluate(\n    X_curr_scaled, y_curr, \n    data_name=\"Current Data (with drift)\", \n    task='classification'\n)\n\n#%% [markdown]\n# ### 2.3 จำลองการ Monitor หลายช่วงเวลา\n\n#%%\ndef simulate_time_series_monitoring(model, scaler, n_periods=10):\n    \"\"\"\n    จำลองการ monitor โมเดลข้ามหลายช่วงเวลา\n    \"\"\"\n    performance_over_time = []\n    \n    for period in range(n_periods):\n        # สร้างข้อมูลจำลองแต่ละช่วงเวลา โดย drift เพิ่มขึ้นเรื่อยๆ\n        drift_level = period * 0.05  # drift เพิ่มขึ้น 5% ต่อ period\n        \n        period_data = create_credit_data(\n            n_samples=500, \n            seed=42 + period,\n            drift_level=drift_level\n        )\n        \n        X_period, y_period = prepare_features(period_data)\n        X_period_scaled = scaler.transform(X_period)\n        \n        # ทำนาย\n        y_pred_period = model.predict(X_period_scaled)\n        \n        # คำนวณ metrics\n        metrics = {\n            'period': period + 1,\n            'drift_level': drift_level,\n            'accuracy': accuracy_score(y_period, y_pred_period),\n            'precision': precision_score(y_period, y_pred_period, average='weighted', zero_division=0),\n            'recall': recall_score(y_period, y_pred_period, average='weighted', zero_division=0),\n            'f1': f1_score(y_period, y_pred_period, average='weighted', zero_division=0),\n            'n_samples': len(y_period),\n            'default_rate': y_period.mean()\n        }\n        \n        performance_over_time.append(metrics)\n    \n    return pd.DataFrame(performance_over_time)\n\n#%%\n# จำลองการ monitor 10 ช่วงเวลา\nprint(\"🔄 จำลองการ Monitor ข้ามหลายช่วงเวลา...\")\ntime_series_perf = simulate_time_series_monitoring(rf_model, scaler, n_periods=10)\nprint(\"\\n📊 Performance Over Time:\")\nprint(time_series_perf.to_string(index=False))\n\n#%% [markdown]\n# ### 2.4 Visualization: Performance Over Time\n\n#%%\ndef plot_performance_over_time(perf_df):\n    \"\"\"\n    แสดงกราฟประสิทธิภาพโมเดลตามเวลา\n    \"\"\"\n    fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n    fig.suptitle('📈 Model Performance Over Time', fontsize=14, fontweight='bold')\n    \n    # 1. Main Metrics Over Time\n    ax1 = axes[0, 0]\n    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']\n    for metric in metrics_to_plot:\n        ax1.plot(perf_df['period'], perf_df[metric], marker='o', label=metric.capitalize())\n    ax1.set_xlabel('Time Period')\n    ax1.set_ylabel('Score')\n    ax1.set_title('Classification Metrics Over Time')\n    ax1.legend()\n    ax1.set_ylim([0.5, 1.0])\n    ax1.axhline(y=0.8, color='red', linestyle='--', alpha=0.5, label='Threshold')\n    ax1.grid(True, alpha=0.3)\n    \n    # 2. Accuracy vs Drift Level\n    ax2 = axes[0, 1]\n    ax2.scatter(perf_df['drift_level'], perf_df['accuracy'], s=100, c=perf_df['period'], cmap='viridis')\n    ax2.plot(perf_df['drift_level'], perf_df['accuracy'], 'r--', alpha=0.5)\n    ax2.set_xlabel('Drift Level')\n    ax2.set_ylabel('Accuracy')\n    ax2.set_title('Accuracy vs Data Drift Level')\n    \n    # เพิ่ม colorbar\n    sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=1, vmax=len(perf_df)))\n    sm.set_array([])\n    cbar = plt.colorbar(sm, ax=ax2)\n    cbar.set_label('Period')\n    \n    # 3. Default Rate Over Time\n    ax3 = axes[1, 0]\n    ax3.bar(perf_df['period'], perf_df['default_rate'], color='coral', alpha=0.7)\n    ax3.set_xlabel('Time Period')\n    ax3.set_ylabel('Default Rate')\n    ax3.set_title('Target (Default Rate) Over Time')\n    ax3.axhline(y=perf_df['default_rate'].iloc[0], color='green', linestyle='--', \n                label=f\"Baseline: {perf_df['default_rate'].iloc[0]:.2%}\")\n    ax3.legend()\n    \n    # 4. Performance Degradation Alert\n    ax4 = axes[1, 1]\n    \n    # คำนวณ degradation จาก baseline\n    baseline_accuracy = perf_df['accuracy'].iloc[0]\n    degradation = (baseline_accuracy - perf_df['accuracy']) / baseline_accuracy * 100\n    \n    colors = ['green' if d < 5 else 'orange' if d < 10 else 'red' for d in degradation]\n    ax4.bar(perf_df['period'], degradation, color=colors, alpha=0.7)\n    ax4.set_xlabel('Time Period')\n    ax4.set_ylabel('Degradation (%)')\n    ax4.set_title('Accuracy Degradation from Baseline')\n    ax4.axhline(y=5, color='orange', linestyle='--', alpha=0.7, label='Warning (5%)')\n    ax4.axhline(y=10, color='red', linestyle='--', alpha=0.7, label='Critical (10%)')\n    ax4.legend()\n    \n    plt.tight_layout()\n    plt.show()\n    \n    # สรุปผล\n    print(\"\\n📊 Performance Summary:\")\n    print(f\"   Baseline Accuracy: {baseline_accuracy:.4f}\")\n    print(f\"   Final Accuracy: {perf_df['accuracy'].iloc[-1]:.4f}\")\n    print(f\"   Total Degradation: {degradation.iloc[-1]:.2f}%\")\n    \n    if degradation.iloc[-1] > 10:\n        print(\"   🔴 Status: CRITICAL - ต้อง retrain โมเดล!\")\n    elif degradation.iloc[-1] > 5:\n        print(\"   🟡 Status: WARNING - ควรตรวจสอบและเตรียม retrain\")\n    else:\n        print(\"   🟢 Status: HEALTHY - โมเดลทำงานปกติ\")\n\n#%%\n# แสดงกราฟ\nplot_performance_over_time(time_series_perf)\n\n#%% [markdown]\n# ### 2.5 Confusion Matrix Visualization\n\n#%%\ndef plot_confusion_matrix_comparison(y_true_baseline, y_pred_baseline, \n                                      y_true_current, y_pred_current):\n    \"\"\"\n    เปรียบเทียบ Confusion Matrix ระหว่าง Baseline และ Current\n    \"\"\"\n    fig, axes = plt.subplots(1, 2, figsize=(12, 5))\n    fig.suptitle('📊 Confusion Matrix Comparison', fontsize=14, fontweight='bold')\n    \n    # Baseline\n    cm_baseline = confusion_matrix(y_true_baseline, y_pred_baseline)\n    sns.heatmap(cm_baseline, annot=True, fmt='d', cmap='Blues', ax=axes[0],\n                xticklabels=['No Default', 'Default'],\n                yticklabels=['No Default', 'Default'])\n    axes[0].set_title('Baseline (Test Data)')\n    axes[0].set_xlabel('Predicted')\n    axes[0].set_ylabel('Actual')\n    \n    # Current\n    cm_current = confusion_matrix(y_true_current, y_pred_current)\n    sns.heatmap(cm_current, annot=True, fmt='d', cmap='Oranges', ax=axes[1],\n                xticklabels=['No Default', 'Default'],\n                yticklabels=['No Default', 'Default'])\n    axes[1].set_title('Current Data (with Drift)')\n    axes[1].set_xlabel('Predicted')\n    axes[1].set_ylabel('Actual')\n    \n    plt.tight_layout()\n    plt.show()\n\n#%%\n# ทำนาย current data\ny_pred_curr = rf_model.predict(X_curr_scaled)\n\n# แสดง confusion matrix comparison\nplot_confusion_matrix_comparison(y_test, y_pred_test, y_curr, y_pred_curr)\n\n#%% [markdown]\n# ---\n# ## 🎯 Section 3: Target Drift Detection\n# ### การตรวจจับการเปลี่ยนแปลงของ Target Distribution\n# \n# **ทฤษฎี:**\n# \n# **Target Drift** (หรือ Concept Drift) คือการเปลี่ยนแปลงของ distribution ของ target variable\n# \n# **ประเภทของ Drift:**\n# 1. **Sudden Drift** - เปลี่ยนแปลงอย่างกะทันหัน\n# 2. **Gradual Drift** - เปลี่ยนแปลงทีละน้อย\n# 3. **Recurring Drift** - เปลี่ยนแปลงเป็นรอบ (เช่น seasonal)\n# \n# **วิธีการตรวจจับ:**\n# 1. **Statistical Tests** - Chi-square, KS test, PSI\n# 2. **Distribution Comparison** - เปรียบเทียบ histogram\n# 3. **Threshold-based Alerts** - แจ้งเตือนเมื่อเกินค่าที่กำหนด\n\n#%% [markdown]\n# ### 3.1 สร้าง Drift Detection Class\n\n#%%\nclass DriftDetector:\n    \"\"\"\n    Class สำหรับตรวจจับ Data Drift และ Target Drift\n    \n    รองรับหลายวิธีการทางสถิติ:\n    - Kolmogorov-Smirnov Test (สำหรับ continuous)\n    - Chi-Square Test (สำหรับ categorical)\n    - Population Stability Index (PSI)\n    - Wasserstein Distance\n    \"\"\"\n    \n    def __init__(self, reference_data, reference_name=\"Reference\"):\n        \"\"\"\n        Parameters:\n        -----------\n        reference_data : DataFrame - ข้อมูล reference\n        reference_name : str - ชื่อข้อมูล reference\n        \"\"\"\n        self.reference_data = reference_data.copy()\n        self.reference_name = reference_name\n        self.drift_results = {}\n        \n    def ks_test(self, reference_col, current_col):\n        \"\"\"\n        Kolmogorov-Smirnov Test สำหรับ continuous variables\n        \n        H0: ทั้งสอง distribution มาจากประชากรเดียวกัน\n        ถ้า p-value < 0.05 แสดงว่ามี drift\n        \"\"\"\n        statistic, p_value = ks_2samp(reference_col.dropna(), current_col.dropna())\n        return {\n            'test': 'Kolmogorov-Smirnov',\n            'statistic': statistic,\n            'p_value': p_value,\n            'drift_detected': p_value < 0.05\n        }\n    \n    def chi_square_test(self, reference_col, current_col):\n        \"\"\"\n        Chi-Square Test สำหรับ categorical variables\n        \"\"\"\n        # สร้าง contingency table\n        ref_counts = reference_col.value_counts()\n        curr_counts = current_col.value_counts()\n        \n        # รวม categories\n        all_categories = set(ref_counts.index) | set(curr_counts.index)\n        \n        ref_freq = [ref_counts.get(cat, 0) for cat in all_categories]\n        curr_freq = [curr_counts.get(cat, 0) for cat in all_categories]\n        \n        contingency = np.array([ref_freq, curr_freq])\n        \n        chi2, p_value, dof, expected = chi2_contingency(contingency)\n        \n        return {\n            'test': 'Chi-Square',\n            'statistic': chi2,\n            'p_value': p_value,\n            'degrees_of_freedom': dof,\n            'drift_detected': p_value < 0.05\n        }\n    \n    def calculate_psi(self, reference_col, current_col, bins=10):\n        \"\"\"\n        Population Stability Index (PSI)\n        \n        PSI < 0.1: ไม่มี drift\n        0.1 <= PSI < 0.25: drift เล็กน้อย\n        PSI >= 0.25: drift มาก\n        \"\"\"\n        # สร้าง bins จาก reference\n        ref_min, ref_max = reference_col.min(), reference_col.max()\n        bin_edges = np.linspace(ref_min, ref_max, bins + 1)\n        \n        # คำนวณ % ในแต่ละ bin\n        ref_hist, _ = np.histogram(reference_col, bins=bin_edges)\n        curr_hist, _ = np.histogram(current_col.clip(ref_min, ref_max), bins=bin_edges)\n        \n        # แปลงเป็น percentage\n        ref_pct = ref_hist / len(reference_col)\n        curr_pct = curr_hist / len(current_col)\n        \n        # หลีกเลี่ยง log(0)\n        ref_pct = np.where(ref_pct == 0, 0.0001, ref_pct)\n        curr_pct = np.where(curr_pct == 0, 0.0001, curr_pct)\n        \n        # คำนวณ PSI\n        psi = np.sum((curr_pct - ref_pct) * np.log(curr_pct / ref_pct))\n        \n        return {\n            'psi': psi,\n            'interpretation': 'No Drift' if psi < 0.1 else 'Slight Drift' if psi < 0.25 else 'Significant Drift',\n            'drift_detected': psi >= 0.1\n        }\n    \n    def wasserstein_distance_test(self, reference_col, current_col):\n        \"\"\"\n        Wasserstein Distance (Earth Mover's Distance)\n        วัดระยะห่างระหว่างสอง distributions\n        \"\"\"\n        distance = wasserstein_distance(reference_col.dropna(), current_col.dropna())\n        \n        # Normalize by reference std\n        ref_std = reference_col.std()\n        normalized_distance = distance / ref_std if ref_std > 0 else distance\n        \n        return {\n            'distance': distance,\n            'normalized_distance': normalized_distance,\n            'drift_detected': normalized_distance > 0.1\n        }\n    \n    def detect_feature_drift(self, current_data, columns=None):\n        \"\"\"\n        ตรวจจับ drift สำหรับทุก features\n        \"\"\"\n        if columns is None:\n            # ใช้ทุก columns ที่มีทั้งสอง datasets\n            columns = [col for col in self.reference_data.columns if col in current_data.columns]\n        \n        results = []\n        \n        for col in columns:\n            ref_col = self.reference_data[col]\n            curr_col = current_data[col]\n            \n            # เลือกวิธีทดสอบตามประเภทข้อมูล\n            if ref_col.dtype in ['object', 'category'] or ref_col.nunique() < 10:\n                # Categorical\n                test_result = self.chi_square_test(ref_col, curr_col)\n                test_type = 'categorical'\n            else:\n                # Continuous\n                ks_result = self.ks_test(ref_col, curr_col)\n                psi_result = self.calculate_psi(ref_col, curr_col)\n                \n                test_result = {\n                    'test': 'KS + PSI',\n                    'ks_statistic': ks_result['statistic'],\n                    'ks_p_value': ks_result['p_value'],\n                    'psi': psi_result['psi'],\n                    'psi_interpretation': psi_result['interpretation'],\n                    'drift_detected': ks_result['drift_detected'] or psi_result['drift_detected']\n                }\n                test_type = 'continuous'\n            \n            results.append({\n                'column': col,\n                'type': test_type,\n                **test_result\n            })\n        \n        self.drift_results['feature_drift'] = results\n        return pd.DataFrame(results)\n    \n    def detect_target_drift(self, current_data, target_col):\n        \"\"\"\n        ตรวจจับ Target Drift โดยเฉพาะ\n        \"\"\"\n        ref_target = self.reference_data[target_col]\n        curr_target = current_data[target_col]\n        \n        result = {\n            'column': target_col,\n            'reference_mean': ref_target.mean(),\n            'current_mean': curr_target.mean(),\n            'reference_std': ref_target.std(),\n            'current_std': curr_target.std()\n        }\n        \n        # ทดสอบทางสถิติ\n        if ref_target.nunique() <= 2:  # Binary target\n            # Chi-square test\n            chi_result = self.chi_square_test(ref_target, curr_target)\n            result.update({\n                'test': 'Chi-Square',\n                'statistic': chi_result['statistic'],\n                'p_value': chi_result['p_value'],\n                'drift_detected': chi_result['drift_detected']\n            })\n        else:\n            # KS test + PSI\n            ks_result = self.ks_test(ref_target, curr_target)\n            psi_result = self.calculate_psi(ref_target, curr_target)\n            result.update({\n                'test': 'KS + PSI',\n                'ks_statistic': ks_result['statistic'],\n                'ks_p_value': ks_result['p_value'],\n                'psi': psi_result['psi'],\n                'drift_detected': ks_result['drift_detected'] or psi_result['drift_detected']\n            })\n        \n        self.drift_results['target_drift'] = result\n        return result\n    \n    def generate_drift_report(self, current_data, target_col=None):\n        \"\"\"\n        สร้าง Drift Report ฉบับเต็ม\n        \"\"\"\n        print(\"=\" * 70)\n        print(\"🔍 DRIFT DETECTION REPORT\")\n        print(\"=\" * 70)\n        print(f\"📅 Generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\")\n        print(f\"📊 Reference Data: {len(self.reference_data):,} samples\")\n        print(f\"📊 Current Data: {len(current_data):,} samples\")\n        print()\n        \n        # Feature Drift\n        print(\"-\" * 50)\n        print(\"1️⃣ FEATURE DRIFT DETECTION\")\n        print(\"-\" * 50)\n        \n        feature_cols = [col for col in self.reference_data.columns \n                       if col != target_col and col in current_data.columns]\n        feature_drift_df = self.detect_feature_drift(current_data, feature_cols)\n        \n        drifted_features = feature_drift_df[feature_drift_df['drift_detected'] == True]\n        \n        if len(drifted_features) > 0:\n            print(f\"⚠️ พบ Drift ใน {len(drifted_features)}/{len(feature_cols)} features:\")\n            for _, row in drifted_features.iterrows():\n                print(f\"   🔴 {row['column']}: {row['test']}\")\n        else:\n            print(\"✅ ไม่พบ Feature Drift ที่มีนัยสำคัญ\")\n        print()\n        \n        # Target Drift\n        if target_col:\n            print(\"-\" * 50)\n            print(\"2️⃣ TARGET DRIFT DETECTION\")\n            print(\"-\" * 50)\n            \n            target_result = self.detect_target_drift(current_data, target_col)\n            \n            print(f\"   Target Column: {target_col}\")\n            print(f\"   Reference Mean: {target_result['reference_mean']:.4f}\")\n            print(f\"   Current Mean: {target_result['current_mean']:.4f}\")\n            print(f\"   Change: {(target_result['current_mean'] - target_result['reference_mean']):.4f}\")\n            print(f\"   Test: {target_result['test']}\")\n            print(f\"   P-value: {target_result.get('p_value', 'N/A')}\")\n            \n            if target_result['drift_detected']:\n                print(\"   🔴 Status: TARGET DRIFT DETECTED!\")\n            else:\n                print(\"   🟢 Status: No significant target drift\")\n        \n        print()\n        print(\"=\" * 70)\n        print(\"📊 END OF DRIFT REPORT\")\n        print(\"=\" * 70)\n        \n        return {\n            'feature_drift': feature_drift_df,\n            'target_drift': self.drift_results.get('target_drift')\n        }\n\n#%%\n# ทดสอบ Drift Detection\nprint(\"🔍 ตรวจจับ Drift ระหว่าง Reference และ Current Data:\")\nprint()\n\ndrift_detector = DriftDetector(reference_data, \"Reference Data\")\ndrift_report = drift_detector.generate_drift_report(current_data, target_col='default')\n\n#%% [markdown]\n# ### 3.2 Visualization: Drift Analysis\n\n#%%\ndef plot_drift_analysis(reference_data, current_data, columns_to_plot=None, target_col='default'):\n    \"\"\"\n    Visualization สำหรับ Drift Analysis\n    \"\"\"\n    if columns_to_plot is None:\n        numeric_cols = reference_data.select_dtypes(include=[np.number]).columns\n        columns_to_plot = [col for col in numeric_cols if col != target_col][:6]\n    \n    n_cols = min(len(columns_to_plot), 3)\n    n_rows = (len(columns_to_plot) + n_cols - 1) // n_cols\n    \n    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))\n    fig.suptitle('📊 Feature Distribution Comparison: Reference vs Current', \n                 fontsize=14, fontweight='bold')\n    \n    if n_rows == 1:\n        axes = [axes] if n_cols == 1 else axes\n    else:\n        axes = axes.flatten()\n    \n    for idx, col in enumerate(columns_to_plot):\n        ax = axes[idx] if isinstance(axes, (list, np.ndarray)) else axes\n        \n        # Plot distributions\n        ax.hist(reference_data[col], bins=30, alpha=0.5, label='Reference', color='blue', density=True)\n        ax.hist(current_data[col], bins=30, alpha=0.5, label='Current', color='orange', density=True)\n        \n        # KS test\n        ks_stat, ks_pval = ks_2samp(reference_data[col].dropna(), current_data[col].dropna())\n        \n        ax.set_title(f'{col}\\nKS stat: {ks_stat:.3f}, p-value: {ks_pval:.3f}')\n        ax.legend()\n        ax.set_xlabel(col)\n        ax.set_ylabel('Density')\n        \n        # เพิ่มสี background ตาม drift status\n        if ks_pval < 0.05:\n            ax.set_facecolor('#ffcccc')  # สีแดงอ่อนถ้ามี drift\n    \n    # ซ่อน axes ที่ไม่ใช้\n    for idx in range(len(columns_to_plot), len(axes)):\n        axes[idx].set_visible(False)\n    \n    plt.tight_layout()\n    plt.show()\n\n#%%\n# แสดง Feature Distribution Comparison\nplot_drift_analysis(reference_data, current_data, target_col='default')\n\n#%%\ndef plot_target_drift(reference_data, current_data, target_col='default'):\n    \"\"\"\n    Visualization สำหรับ Target Drift\n    \"\"\"\n    fig, axes = plt.subplots(1, 3, figsize=(15, 5))\n    fig.suptitle('🎯 Target Drift Analysis', fontsize=14, fontweight='bold')\n    \n    # 1. Distribution Comparison\n    ax1 = axes[0]\n    ref_counts = reference_data[target_col].value_counts(normalize=True)\n    curr_counts = current_data[target_col].value_counts(normalize=True)\n    \n    x = np.arange(len(ref_counts))\n    width = 0.35\n    \n    ax1.bar(x - width/2, ref_counts.values, width, label='Reference', color='blue', alpha=0.7)\n    ax1.bar(x + width/2, curr_counts.values, width, label='Current', color='orange', alpha=0.7)\n    \n    ax1.set_xlabel('Target Value')\n    ax1.set_ylabel('Proportion')\n    ax1.set_title('Target Distribution Comparison')\n    ax1.set_xticks(x)\n    ax1.set_xticklabels(['No Default (0)', 'Default (1)'])\n    ax1.legend()\n    \n    # เพิ่ม % labels\n    for i, (ref_v, curr_v) in enumerate(zip(ref_counts.values, curr_counts.values)):\n        ax1.text(i - width/2, ref_v + 0.01, f'{ref_v:.1%}', ha='center', fontsize=9)\n        ax1.text(i + width/2, curr_v + 0.01, f'{curr_v:.1%}', ha='center', fontsize=9)\n    \n    # 2. Cumulative Distribution\n    ax2 = axes[1]\n    ref_sorted = np.sort(reference_data[target_col])\n    curr_sorted = np.sort(current_data[target_col])\n    \n    ref_cdf = np.arange(1, len(ref_sorted) + 1) / len(ref_sorted)\n    curr_cdf = np.arange(1, len(curr_sorted) + 1) / len(curr_sorted)\n    \n    ax2.plot(ref_sorted, ref_cdf, label='Reference', color='blue')\n    ax2.plot(curr_sorted, curr_cdf, label='Current', color='orange')\n    ax2.set_xlabel('Target Value')\n    ax2.set_ylabel('Cumulative Probability')\n    ax2.set_title('Cumulative Distribution Function (CDF)')\n    ax2.legend()\n    ax2.grid(True, alpha=0.3)\n    \n    # 3. Drift Summary\n    ax3 = axes[2]\n    ax3.axis('off')\n    \n    # คำนวณ metrics\n    ref_mean = reference_data[target_col].mean()\n    curr_mean = current_data[target_col].mean()\n    change = curr_mean - ref_mean\n    change_pct = (change / ref_mean) * 100 if ref_mean != 0 else 0\n    \n    # Chi-square test\n    contingency = np.array([\n        [reference_data[target_col].sum(), len(reference_data) - reference_data[target_col].sum()],\n        [current_data[target_col].sum(), len(current_data) - current_data[target_col].sum()]\n    ])\n    chi2, p_value, _, _ = chi2_contingency(contingency)\n    \n    summary_text = f\"\"\"\n    📊 TARGET DRIFT SUMMARY\n    ─────────────────────────────\n    \n    Reference Default Rate: {ref_mean:.2%}\n    Current Default Rate:   {curr_mean:.2%}\n    \n    Absolute Change: {change:+.2%}\n    Relative Change: {change_pct:+.1f}%\n    \n    ─────────────────────────────\n    Statistical Test (Chi-Square):\n    Chi² Statistic: {chi2:.4f}\n    P-value: {p_value:.4f}\n    \n    ─────────────────────────────\n    Status: {'🔴 DRIFT DETECTED' if p_value < 0.05 else '🟢 NO SIGNIFICANT DRIFT'}\n    \"\"\"\n    \n    ax3.text(0.1, 0.5, summary_text, transform=ax3.transAxes, \n             fontsize=11, verticalalignment='center', fontfamily='monospace',\n             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))\n    \n    plt.tight_layout()\n    plt.show()\n\n#%%\n# แสดง Target Drift Analysis\nplot_target_drift(reference_data, current_data, target_col='default')\n\n#%% [markdown]\n# ### 3.3 Prediction Drift Detection\n\n#%%\ndef detect_prediction_drift(model, scaler, reference_data, current_data, target_col='default'):\n    \"\"\"\n    ตรวจจับ Prediction Drift\n    เปรียบเทียบการกระจายของ predictions ระหว่าง reference และ current\n    \"\"\"\n    # เตรียมข้อมูล\n    X_ref, _ = prepare_features(reference_data, target_col)\n    X_curr, _ = prepare_features(current_data, target_col)\n    \n    X_ref_scaled = scaler.transform(X_ref)\n    X_curr_scaled = scaler.transform(X_curr)\n    \n    # ทำนาย probability\n    ref_proba = model.predict_proba(X_ref_scaled)[:, 1]\n    curr_proba = model.predict_proba(X_curr_scaled)[:, 1]\n    \n    # ทำนาย class\n    ref_pred = model.predict(X_ref_scaled)\n    curr_pred = model.predict(X_curr_scaled)\n    \n    # Statistical tests\n    ks_stat, ks_pval = ks_2samp(ref_proba, curr_proba)\n    wasserstein = wasserstein_distance(ref_proba, curr_proba)\n    \n    # Prediction distribution\n    ref_positive_rate = ref_pred.mean()\n    curr_positive_rate = curr_pred.mean()\n    \n    print(\"=\" * 70)\n    print(\"🔮 PREDICTION DRIFT ANALYSIS\")\n    print(\"=\" * 70)\n    print()\n    print(\"📊 Prediction Probability Statistics:\")\n    print(f\"   Reference - Mean: {ref_proba.mean():.4f}, Std: {ref_proba.std():.4f}\")\n    print(f\"   Current   - Mean: {curr_proba.mean():.4f}, Std: {curr_proba.std():.4f}\")\n    print()\n    print(\"📊 Predicted Positive Rate:\")\n    print(f\"   Reference: {ref_positive_rate:.2%}\")\n    print(f\"   Current:   {curr_positive_rate:.2%}\")\n    print(f\"   Change:    {(curr_positive_rate - ref_positive_rate):.2%}\")\n    print()\n    print(\"📊 Statistical Tests:\")\n    print(f\"   KS Statistic: {ks_stat:.4f}\")\n    print(f\"   KS P-value: {ks_pval:.4f}\")\n    print(f\"   Wasserstein Distance: {wasserstein:.4f}\")\n    print()\n    \n    if ks_pval < 0.05:\n        print(\"   🔴 Status: PREDICTION DRIFT DETECTED!\")\n    else:\n        print(\"   🟢 Status: No significant prediction drift\")\n    \n    print(\"=\" * 70)\n    \n    # Visualization\n    fig, axes = plt.subplots(1, 2, figsize=(14, 5))\n    fig.suptitle('🔮 Prediction Drift Analysis', fontsize=14, fontweight='bold')\n    \n    # 1. Probability Distribution\n    ax1 = axes[0]\n    ax1.hist(ref_proba, bins=50, alpha=0.5, label='Reference', color='blue', density=True)\n    ax1.hist(curr_proba, bins=50, alpha=0.5, label='Current', color='orange', density=True)\n    ax1.set_xlabel('Predicted Probability')\n    ax1.set_ylabel('Density')\n    ax1.set_title(f'Prediction Probability Distribution\\nKS stat: {ks_stat:.3f}, p-value: {ks_pval:.4f}')\n    ax1.legend()\n    ax1.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Threshold')\n    \n    # 2. Predicted Class Distribution\n    ax2 = axes[1]\n    categories = ['No Default', 'Default']\n    ref_dist = [(ref_pred == 0).sum(), (ref_pred == 1).sum()]\n    curr_dist = [(curr_pred == 0).sum(), (curr_pred == 1).sum()]\n    \n    x = np.arange(len(categories))\n    width = 0.35\n    \n    ax2.bar(x - width/2, ref_dist, width, label='Reference', color='blue', alpha=0.7)\n    ax2.bar(x + width/2, curr_dist, width, label='Current', color='orange', alpha=0.7)\n    ax2.set_xlabel('Predicted Class')\n    ax2.set_ylabel('Count')\n    ax2.set_title('Predicted Class Distribution')\n    ax2.set_xticks(x)\n    ax2.set_xticklabels(categories)\n    ax2.legend()\n    \n    plt.tight_layout()\n    plt.show()\n    \n    return {\n        'ks_statistic': ks_stat,\n        'ks_pvalue': ks_pval,\n        'wasserstein_distance': wasserstein,\n        'ref_positive_rate': ref_positive_rate,\n        'curr_positive_rate': curr_positive_rate,\n        'drift_detected': ks_pval < 0.05\n    }\n\n#%%\n# ตรวจจับ Prediction Drift\nprediction_drift = detect_prediction_drift(rf_model, scaler, reference_data, current_data)\n\n#%% [markdown]\n# ### 3.4 Drift Alert System\n\n#%%\nclass DriftAlertSystem:\n    \"\"\"\n    ระบบแจ้งเตือน Drift\n    \"\"\"\n    \n    def __init__(self):\n        self.thresholds = {\n            'psi': 0.1,              # PSI threshold\n            'ks_pvalue': 0.05,       # KS test p-value\n            'target_change': 0.05,   # Target rate change\n            'prediction_change': 0.05  # Prediction rate change\n        }\n        self.alerts = []\n    \n    def check_drift_alerts(self, drift_results, data_name=\"Current Data\"):\n        \"\"\"\n        ตรวจสอบและสร้าง alerts จากผลการตรวจจับ drift\n        \"\"\"\n        self.alerts = []\n        \n        # Check feature drift\n        if 'feature_drift' in drift_results:\n            feature_df = drift_results['feature_drift']\n            drifted_features = feature_df[feature_df['drift_detected'] == True]\n            \n            if len(drifted_features) > 0:\n                drift_pct = len(drifted_features) / len(feature_df) * 100\n                alert_type = 'CRITICAL' if drift_pct > 30 else 'WARNING'\n                \n                self.alerts.append({\n                    'type': alert_type,\n                    'category': 'Feature Drift',\n                    'message': f\"พบ {len(drifted_features)} features ({drift_pct:.1f}%) มี drift\",\n                    'details': drifted_features['column'].tolist()\n                })\n        \n        # Check target drift\n        if 'target_drift' in drift_results and drift_results['target_drift']:\n            target_result = drift_results['target_drift']\n            if target_result.get('drift_detected', False):\n                change = abs(target_result['current_mean'] - target_result['reference_mean'])\n                \n                self.alerts.append({\n                    'type': 'CRITICAL' if change > 0.1 else 'WARNING',\n                    'category': 'Target Drift',\n                    'message': f\"Target distribution เปลี่ยนแปลง (change: {change:.2%})\",\n                    'details': target_result\n                })\n        \n        # Display alerts\n        self._display_alerts(data_name)\n        \n        return self.alerts\n    \n    def _display_alerts(self, data_name):\n        \"\"\"แสดงผล alerts\"\"\"\n        print(\"=\" * 70)\n        print(f\"🚨 DRIFT ALERTS: {data_name}\")\n        print(\"=\" * 70)\n        \n        if not self.alerts:\n            print(\"✅ ไม่มี drift alerts - ข้อมูลยังคงมีลักษณะใกล้เคียงกับ reference\")\n        else:\n            print(f\"⚠️ พบ {len(self.alerts)} alerts:\")\n            print()\n            \n            for i, alert in enumerate(self.alerts, 1):\n                icon = \"🔴\" if alert['type'] == 'CRITICAL' else \"🟡\"\n                print(f\"{icon} Alert #{i}: [{alert['type']}] {alert['category']}\")\n                print(f\"   Message: {alert['message']}\")\n                \n                if isinstance(alert['details'], list):\n                    print(f\"   Affected: {', '.join(alert['details'][:5])}\")\n                    if len(alert['details']) > 5:\n                        print(f\"            ... and {len(alert['details']) - 5} more\")\n                print()\n        \n        print(\"=\" * 70)\n\n#%%\n# ทดสอบ Drift Alert System\ndrift_alert = DriftAlertSystem()\nalerts = drift_alert.check_drift_alerts(drift_report, \"Current Data\")\n\n#%% [markdown]\n# ---\n# ## 📊 Section 4: Building Monitoring Dashboard\n# ### การสร้าง Dashboard สำหรับ Monitoring\n# \n# **ทฤษฎี:**\n# \n# Monitoring Dashboard รวมข้อมูลจากหลายแหล่งเพื่อให้เห็นภาพรวมของสถานะโมเดล\n# \n# **องค์ประกอบสำคัญ:**\n# 1. Data Quality Summary\n# 2. Model Performance Metrics\n# 3. Drift Detection Results\n# 4. Alert Summary\n# 5. Historical Trends\n\n#%% [markdown]\n# ### 4.1 สร้าง Comprehensive Dashboard Class\n\n#%%\nclass ModelMonitoringDashboard:\n    \"\"\"\n    Dashboard สำหรับ Model Monitoring\n    รวม Data Quality, Performance, และ Drift Detection\n    \"\"\"\n    \n    def __init__(self, model, model_name=\"ML Model\"):\n        self.model = model\n        self.model_name = model_name\n        self.reports = {\n            'data_quality': {},\n            'performance': {},\n            'drift': {},\n            'alerts': []\n        }\n        self.history = []\n    \n    def run_full_monitoring(self, reference_data, current_data, target_col, scaler=None):\n        \"\"\"\n        รัน monitoring ทั้งหมด\n        \"\"\"\n        print(\"=\" * 80)\n        print(\"🖥️  MODEL MONITORING DASHBOARD\")\n        print(f\"   Model: {self.model_name}\")\n        print(f\"   Time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\")\n        print(\"=\" * 80)\n        print()\n        \n        # 1. Data Quality Monitoring\n        print(\"📊 [1/4] Running Data Quality Check...\")\n        dq_monitor = DataQualityMonitor(current_data, \"Current Data\")\n        self.reports['data_quality'] = {\n            'missing': dq_monitor.check_missing_values(),\n            'duplicates': dq_monitor.check_duplicates(),\n            'outliers': dq_monitor.detect_outliers()\n        }\n        \n        # 2. Performance Monitoring\n        print(\"📈 [2/4] Running Performance Evaluation...\")\n        if scaler is not None:\n            X_ref, y_ref = prepare_features(reference_data, target_col)\n            X_curr, y_curr = prepare_features(current_data, target_col)\n            \n            X_ref_scaled = scaler.transform(X_ref)\n            X_curr_scaled = scaler.transform(X_curr)\n            \n            y_pred_ref = self.model.predict(X_ref_scaled)\n            y_pred_curr = self.model.predict(X_curr_scaled)\n            \n            self.reports['performance'] = {\n                'reference': {\n                    'accuracy': accuracy_score(y_ref, y_pred_ref),\n                    'precision': precision_score(y_ref, y_pred_ref, average='weighted', zero_division=0),\n                    'recall': recall_score(y_ref, y_pred_ref, average='weighted', zero_division=0),\n                    'f1': f1_score(y_ref, y_pred_ref, average='weighted', zero_division=0)\n                },\n                'current': {\n                    'accuracy': accuracy_score(y_curr, y_pred_curr),\n                    'precision': precision_score(y_curr, y_pred_curr, average='weighted', zero_division=0),\n                    'recall': recall_score(y_curr, y_pred_curr, average='weighted', zero_division=0),\n                    'f1': f1_score(y_curr, y_pred_curr, average='weighted', zero_division=0)\n                }\n            }\n        \n        # 3. Drift Detection\n        print(\"🔍 [3/4] Running Drift Detection...\")\n        drift_detector = DriftDetector(reference_data)\n        feature_drift = drift_detector.detect_feature_drift(current_data, \n            [col for col in reference_data.columns if col != target_col and col in current_data.columns])\n        target_drift = drift_detector.detect_target_drift(current_data, target_col)\n        \n        self.reports['drift'] = {\n            'feature_drift': feature_drift,\n            'target_drift': target_drift\n        }\n        \n        # 4. Generate Alerts\n        print(\"🚨 [4/4] Generating Alerts...\")\n        self._generate_alerts()\n        \n        print()\n        print(\"✅ Monitoring Complete!\")\n        print()\n        \n        # Save to history\n        self.history.append({\n            'timestamp': datetime.now(),\n            'reports': self.reports.copy()\n        })\n        \n        return self.reports\n    \n    def _generate_alerts(self):\n        \"\"\"สร้าง alerts จากผลการ monitoring\"\"\"\n        self.reports['alerts'] = []\n        \n        # Data Quality Alerts\n        dq = self.reports['data_quality']\n        if len(dq.get('missing', pd.DataFrame())) > 0:\n            self.reports['alerts'].append({\n                'type': 'WARNING',\n                'category': 'Data Quality',\n                'message': 'พบ missing values ในข้อมูล'\n            })\n        \n        dup = dq.get('duplicates', {})\n        if dup.get('duplicate_rows', 0) > 0:\n            self.reports['alerts'].append({\n                'type': 'WARNING',\n                'category': 'Data Quality',\n                'message': f\"พบ {dup['duplicate_rows']} duplicate rows\"\n            })\n        \n        # Performance Alerts\n        perf = self.reports.get('performance', {})\n        if perf:\n            ref_acc = perf['reference']['accuracy']\n            curr_acc = perf['current']['accuracy']\n            degradation = (ref_acc - curr_acc) / ref_acc * 100\n            \n            if degradation > 10:\n                self.reports['alerts'].append({\n                    'type': 'CRITICAL',\n                    'category': 'Performance',\n                    'message': f\"Accuracy ลดลง {degradation:.1f}% จาก baseline\"\n                })\n            elif degradation > 5:\n                self.reports['alerts'].append({\n                    'type': 'WARNING',\n                    'category': 'Performance',\n                    'message': f\"Accuracy ลดลง {degradation:.1f}% จาก baseline\"\n                })\n        \n        # Drift Alerts\n        drift = self.reports.get('drift', {})\n        if 'target_drift' in drift and drift['target_drift'].get('drift_detected', False):\n            self.reports['alerts'].append({\n                'type': 'CRITICAL',\n                'category': 'Drift',\n                'message': 'ตรวจพบ Target Drift'\n            })\n        \n        if 'feature_drift' in drift:\n            drifted = drift['feature_drift'][drift['feature_drift']['drift_detected'] == True]\n            if len(drifted) > 0:\n                self.reports['alerts'].append({\n                    'type': 'WARNING',\n                    'category': 'Drift',\n                    'message': f\"พบ Feature Drift ใน {len(drifted)} features\"\n                })\n    \n    def display_dashboard(self):\n        \"\"\"แสดง Dashboard\"\"\"\n        print()\n        print(\"╔\" + \"═\" * 78 + \"╗\")\n        print(\"║\" + \" \" * 25 + \"📊 MONITORING DASHBOARD\" + \" \" * 30 + \"║\")\n        print(\"╠\" + \"═\" * 78 + \"╣\")\n        \n        # Overall Status\n        n_critical = sum(1 for a in self.reports['alerts'] if a['type'] == 'CRITICAL')\n        n_warning = sum(1 for a in self.reports['alerts'] if a['type'] == 'WARNING')\n        \n        if n_critical > 0:\n            status = \"🔴 CRITICAL\"\n            status_color = \"Action Required!\"\n        elif n_warning > 0:\n            status = \"🟡 WARNING\"\n            status_color = \"Attention Needed\"\n        else:\n            status = \"🟢 HEALTHY\"\n            status_color = \"All Systems Normal\"\n        \n        print(f\"║  Overall Status: {status} - {status_color}\")\n        print(f\"║  Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\")\n        print(\"╠\" + \"═\" * 78 + \"╣\")\n        \n        # Performance Summary\n        print(\"║  📈 PERFORMANCE SUMMARY\")\n        print(\"║  \" + \"-\" * 40)\n        perf = self.reports.get('performance', {})\n        if perf:\n            print(f\"║       Metric      │ Reference │ Current │ Change\")\n            print(\"║  \" + \"-\" * 50)\n            for metric in ['accuracy', 'precision', 'recall', 'f1']:\n                ref_val = perf['reference'].get(metric, 0)\n                curr_val = perf['current'].get(metric, 0)\n                change = curr_val - ref_val\n                arrow = \"↑\" if change > 0 else \"↓\" if change < 0 else \"→\"\n                print(f\"║    {metric:12} │  {ref_val:.4f}  │ {curr_val:.4f} │ {arrow} {abs(change):.4f}\")\n        print(\"║\")\n        \n        # Alerts Summary\n        print(\"╠\" + \"═\" * 78 + \"╣\")\n        print(\"║  🚨 ALERTS SUMMARY\")\n        print(\"║  \" + \"-\" * 40)\n        \n        if not self.reports['alerts']:\n            print(\"║    ✅ No alerts - All metrics within acceptable range\")\n        else:\n            for alert in self.reports['alerts']:\n                icon = \"🔴\" if alert['type'] == 'CRITICAL' else \"🟡\"\n                print(f\"║    {icon} [{alert['type']}] {alert['category']}: {alert['message']}\")\n        \n        print(\"║\")\n        print(\"╚\" + \"═\" * 78 + \"╝\")\n    \n    def plot_dashboard(self):\n        \"\"\"สร้าง Visual Dashboard\"\"\"\n        fig = plt.figure(figsize=(16, 12))\n        fig.suptitle(f'📊 Model Monitoring Dashboard: {self.model_name}', \n                     fontsize=16, fontweight='bold')\n        \n        # Create grid\n        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)\n        \n        # 1. Overall Status (top left)\n        ax1 = fig.add_subplot(gs[0, 0])\n        n_critical = sum(1 for a in self.reports['alerts'] if a['type'] == 'CRITICAL')\n        n_warning = sum(1 for a in self.reports['alerts'] if a['type'] == 'WARNING')\n        \n        if n_critical > 0:\n            status_color = 'red'\n            status_text = 'CRITICAL'\n        elif n_warning > 0:\n            status_color = 'orange'\n            status_text = 'WARNING'\n        else:\n            status_color = 'green'\n            status_text = 'HEALTHY'\n        \n        ax1.pie([1], colors=[status_color], startangle=90)\n        circle = plt.Circle((0, 0), 0.7, fc='white')\n        ax1.add_patch(circle)\n        ax1.text(0, 0, status_text, ha='center', va='center', fontsize=14, fontweight='bold')\n        ax1.set_title('System Status')\n        \n        # 2. Performance Comparison (top middle and right)\n        ax2 = fig.add_subplot(gs[0, 1:])\n        perf = self.reports.get('performance', {})\n        if perf:\n            metrics = ['accuracy', 'precision', 'recall', 'f1']\n            ref_values = [perf['reference'].get(m, 0) for m in metrics]\n            curr_values = [perf['current'].get(m, 0) for m in metrics]\n            \n            x = np.arange(len(metrics))\n            width = 0.35\n            \n            ax2.bar(x - width/2, ref_values, width, label='Reference', color='steelblue', alpha=0.8)\n            ax2.bar(x + width/2, curr_values, width, label='Current', color='coral', alpha=0.8)\n            ax2.set_xticks(x)\n            ax2.set_xticklabels([m.capitalize() for m in metrics])\n            ax2.set_ylabel('Score')\n            ax2.set_title('Performance Metrics Comparison')\n            ax2.legend()\n            ax2.set_ylim([0, 1.1])\n            ax2.axhline(y=0.8, color='red', linestyle='--', alpha=0.5)\n        \n        # 3. Data Quality (middle left)\n        ax3 = fig.add_subplot(gs[1, 0])\n        dq = self.reports['data_quality']\n        \n        quality_scores = []\n        quality_labels = []\n        \n        # Missing score\n        missing_df = dq.get('missing', pd.DataFrame())\n        missing_score = 100 if len(missing_df) == 0 else max(0, 100 - len(missing_df) * 10)\n        quality_scores.append(missing_score)\n        quality_labels.append('Missing\\nValues')\n        \n        # Duplicate score\n        dup = dq.get('duplicates', {})\n        dup_pct = dup.get('duplicate_percentage', 0)\n        dup_score = max(0, 100 - dup_pct * 10)\n        quality_scores.append(dup_score)\n        quality_labels.append('Duplicates')\n        \n        # Outlier score\n        outlier_df = dq.get('outliers', pd.DataFrame())\n        if len(outlier_df) > 0:\n            avg_outlier_pct = outlier_df['outlier_percentage'].mean()\n            outlier_score = max(0, 100 - avg_outlier_pct * 5)\n        else:\n            outlier_score = 100\n        quality_scores.append(outlier_score)\n        quality_labels.append('Outliers')\n        \n        colors = ['green' if s >= 80 else 'orange' if s >= 60 else 'red' for s in quality_scores]\n        ax3.barh(quality_labels, quality_scores, color=colors, alpha=0.7)\n        ax3.set_xlim([0, 100])\n        ax3.set_title('Data Quality Scores')\n        ax3.axvline(x=80, color='green', linestyle='--', alpha=0.5)\n        \n        # 4. Drift Detection Summary (middle center and right)\n        ax4 = fig.add_subplot(gs[1, 1:])\n        drift = self.reports.get('drift', {})\n        \n        if 'feature_drift' in drift:\n            feature_df = drift['feature_drift']\n            n_features = len(feature_df)\n            n_drifted = feature_df['drift_detected'].sum()\n            \n            sizes = [n_drifted, n_features - n_drifted]\n            labels = [f'Drifted ({n_drifted})', f'Stable ({n_features - n_drifted})']\n            colors_pie = ['coral', 'steelblue']\n            explode = (0.05, 0)\n            \n            ax4.pie(sizes, labels=labels, colors=colors_pie, autopct='%1.1f%%', \n                   explode=explode, startangle=90)\n            ax4.set_title('Feature Drift Summary')\n        \n        # 5. Alerts List (bottom)\n        ax5 = fig.add_subplot(gs[2, :])\n        ax5.axis('off')\n        \n        alerts_text = \"🚨 ALERTS:\\n\" + \"-\" * 50 + \"\\n\"\n        if not self.reports['alerts']:\n            alerts_text += \"✅ No alerts - All systems operating normally\"\n        else:\n            for alert in self.reports['alerts']:\n                icon = \"🔴\" if alert['type'] == 'CRITICAL' else \"🟡\"\n                alerts_text += f\"{icon} [{alert['type']}] {alert['category']}: {alert['message']}\\n\"\n        \n        ax5.text(0.1, 0.5, alerts_text, transform=ax5.transAxes,\n                fontsize=11, verticalalignment='center', fontfamily='monospace',\n                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))\n        \n        plt.tight_layout()\n        plt.show()\n    \n    def export_report(self, filename=None):\n        \"\"\"Export report เป็น dictionary\"\"\"\n        if filename is None:\n            filename = f\"monitoring_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}\"\n        \n        report = {\n            'model_name': self.model_name,\n            'timestamp': datetime.now().isoformat(),\n            'data_quality': {\n                'missing_columns': len(self.reports['data_quality'].get('missing', pd.DataFrame())),\n                'duplicates': self.reports['data_quality'].get('duplicates', {})\n            },\n            'performance': self.reports.get('performance', {}),\n            'drift': {\n                'target_drift_detected': self.reports.get('drift', {}).get('target_drift', {}).get('drift_detected', False),\n                'feature_drift_count': len(self.reports.get('drift', {}).get('feature_drift', pd.DataFrame())[\n                    self.reports.get('drift', {}).get('feature_drift', pd.DataFrame()).get('drift_detected', False) == True\n                ]) if 'feature_drift' in self.reports.get('drift', {}) else 0\n            },\n            'alerts': self.reports['alerts'],\n            'overall_status': 'CRITICAL' if any(a['type'] == 'CRITICAL' for a in self.reports['alerts']) \n                             else 'WARNING' if any(a['type'] == 'WARNING' for a in self.reports['alerts'])\n                             else 'HEALTHY'\n        }\n        \n        print(f\"📄 Report exported: {filename}\")\n        return report\n\n#%%\n# สร้างและรัน Dashboard\nprint(\"🖥️ สร้าง Monitoring Dashboard...\")\ndashboard = ModelMonitoringDashboard(rf_model, \"Credit Risk Random Forest\")\n\n# รัน Full Monitoring\nreports = dashboard.run_full_monitoring(\n    reference_data=reference_data,\n    current_data=current_data,\n    target_col='default',\n    scaler=scaler\n)\n\n#%%\n# แสดง Text Dashboard\ndashboard.display_dashboard()\n\n#%%\n# แสดง Visual Dashboard\ndashboard.plot_dashboard()\n\n#%%\n# Export Report\nexported_report = dashboard.export_report()\nprint(\"\\n📋 Exported Report Summary:\")\nfor key, value in exported_report.items():\n    if key != 'alerts':\n        print(f\"   {key}: {value}\")\n\n#%% [markdown]\n# ---\n# ## 🎓 Section 5: สรุปและแนวทางปฏิบัติ\n# ### Summary and Best Practices\n\n#%% [markdown]\n# ### 5.1 สรุปสิ่งที่เรียนรู้\n# \n# ใน LAB นี้ เราได้เรียนรู้เกี่ยวกับ Model Monitoring ดังนี้:\n# \n# #### 1. Data Quality Monitoring\n# - ตรวจสอบ missing values, duplicates, outliers\n# - สร้าง quality scores และ alerts\n# - ใช้ threshold-based monitoring\n# \n# #### 2. Model Performance Tracking\n# - ติดตาม classification metrics (Accuracy, Precision, Recall, F1)\n# - เปรียบเทียบ baseline vs current performance\n# - ตรวจจับ performance degradation\n# \n# #### 3. Drift Detection\n# - ใช้ statistical tests (KS, Chi-square, PSI)\n# - ตรวจจับ feature drift และ target drift\n# - สร้าง prediction drift analysis\n# \n# #### 4. Monitoring Dashboard\n# - รวม metrics ทั้งหมดใน single view\n# - สร้าง alerts และ recommendations\n# - Export reports สำหรับ stakeholders\n\n#%% [markdown]\n# ### 5.2 Best Practices สำหรับ Model Monitoring\n# \n# 1. **กำหนด Baseline ที่ชัดเจน**\n#    - บันทึก performance metrics เมื่อ deploy\n#    - เก็บ reference data distribution\n# \n# 2. **Monitor อย่างสม่ำเสมอ**\n#    - ตั้ง schedule สำหรับ monitoring (รายวัน/รายสัปดาห์)\n#    - Automate monitoring pipeline\n# \n# 3. **ตั้ง Threshold ที่เหมาะสม**\n#    - ไม่ sensitive เกินไป (false alarms)\n#    - ไม่ loose เกินไป (miss real issues)\n# \n# 4. **มี Action Plan**\n#    - กำหนดว่าจะทำอะไรเมื่อเกิด alert\n#    - Retrain strategy เมื่อ performance ต่ำกว่าเกณฑ์\n# \n# 5. **Document Everything**\n#    - บันทึก monitoring results\n#    - Track model versions และ changes\n\n#%%\n# สรุป Code สำหรับ Quick Monitoring\ndef quick_model_monitor(model, scaler, reference_data, current_data, target_col):\n    \"\"\"\n    ฟังก์ชันสำหรับ quick monitoring\n    ใช้สำหรับตรวจสอบโมเดลอย่างรวดเร็ว\n    \"\"\"\n    print(\"🚀 Quick Model Monitoring\")\n    print(\"=\" * 50)\n    \n    # Prepare data\n    X_ref, y_ref = prepare_features(reference_data, target_col)\n    X_curr, y_curr = prepare_features(current_data, target_col)\n    \n    X_ref_scaled = scaler.transform(X_ref)\n    X_curr_scaled = scaler.transform(X_curr)\n    \n    # Predictions\n    y_pred_ref = model.predict(X_ref_scaled)\n    y_pred_curr = model.predict(X_curr_scaled)\n    \n    # Calculate metrics\n    ref_acc = accuracy_score(y_ref, y_pred_ref)\n    curr_acc = accuracy_score(y_curr, y_pred_curr)\n    degradation = (ref_acc - curr_acc) / ref_acc * 100\n    \n    # Target drift\n    ref_rate = y_ref.mean()\n    curr_rate = y_curr.mean()\n    rate_change = abs(curr_rate - ref_rate)\n    \n    # Print summary\n    print(f\"📊 Reference Accuracy: {ref_acc:.4f}\")\n    print(f\"📊 Current Accuracy:   {curr_acc:.4f}\")\n    print(f\"📉 Degradation:        {degradation:.2f}%\")\n    print()\n    print(f\"🎯 Reference Target Rate: {ref_rate:.2%}\")\n    print(f\"🎯 Current Target Rate:   {curr_rate:.2%}\")\n    print(f\"📈 Rate Change:           {rate_change:.2%}\")\n    print()\n    \n    # Status\n    if degradation > 10 or rate_change > 0.1:\n        print(\"🔴 Status: CRITICAL - Consider retraining!\")\n    elif degradation > 5 or rate_change > 0.05:\n        print(\"🟡 Status: WARNING - Monitor closely\")\n    else:\n        print(\"🟢 Status: HEALTHY - Model performing well\")\n    \n    print(\"=\" * 50)\n    \n    return {\n        'reference_accuracy': ref_acc,\n        'current_accuracy': curr_acc,\n        'degradation_pct': degradation,\n        'reference_target_rate': ref_rate,\n        'current_target_rate': curr_rate,\n        'rate_change': rate_change\n    }\n\n#%%\n# ทดสอบ Quick Monitor\nprint(\"\\n📊 ทดสอบ Quick Model Monitor:\")\nquick_results = quick_model_monitor(rf_model, scaler, reference_data, current_data, 'default')\n\n#%% [markdown]\n# ### 5.3 แบบฝึกหัด (Exercises)\n# \n# ลองทำแบบฝึกหัดต่อไปนี้เพื่อทดสอบความเข้าใจ:\n# \n# 1. **Exercise 1**: เพิ่ม drift level เป็น 0.5 และ 0.7 แล้วดูผลกระทบต่อ performance\n# \n# 2. **Exercise 2**: ปรับ threshold ของ alerts และดูว่า alerts เปลี่ยนแปลงอย่างไร\n# \n# 3. **Exercise 3**: เพิ่ม metric ใหม่ เช่น AUC-ROC ใน performance monitoring\n# \n# 4. **Exercise 4**: สร้าง monitoring สำหรับ regression model แทน classification\n\n#%%\n# Exercise 1: ทดสอบกับ drift level ที่สูงขึ้น\nprint(\"📝 Exercise 1: ทดสอบกับ High Drift Data\")\nprint()\n\nhigh_drift_data = create_credit_data(n_samples=1000, seed=456, drift_level=0.7)\nprint(\"High Drift Data (drift_level=0.7):\")\nquick_results_high = quick_model_monitor(rf_model, scaler, reference_data, high_drift_data, 'default')\n\n#%% [markdown]\n# ---\n# ## 🎉 จบ LAB: Model Monitoring with Scikit-Learn\n# \n# ### สิ่งที่ได้เรียนรู้:\n# \n# 1. ✅ Data Quality Monitoring\n#    - ตรวจสอบ missing values, duplicates, outliers\n#    - สร้าง quality reports และ alerts\n# \n# 2. ✅ Model Performance Tracking\n#    - ติดตาม classification/regression metrics\n#    - เปรียบเทียบ reference vs current performance\n# \n# 3. ✅ Drift Detection\n#    - ใช้ statistical tests (KS, Chi-square, PSI)\n#    - ตรวจจับ feature drift และ target drift\n# \n# 4. ✅ Monitoring Dashboard\n#    - สร้าง comprehensive dashboard\n#    - Generate alerts และ reports\n# \n# ### ขั้นตอนถัดไป:\n# \n# - นำ monitoring ไปใช้กับ production model\n# - ตั้ง automated monitoring pipeline\n# - เชื่อมต่อกับ alerting system (email, Slack)\n# - สร้าง retraining pipeline เมื่อ performance ต่ำ\n\n#%%\nprint(\"🎉 จบ LAB: Model Monitoring with Scikit-Learn\")\nprint()\nprint(\"ขอบคุณที่เรียนรู้ไปด้วยกัน!\")\nprint(\"Happy Monitoring! 🚀\")"
    }
  ],
  
}


In [4]:
# Simple extraction
text = response["content"][0]["text"]
print(text)

# Save to file
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)

#%% [markdown]
# 🔬 LAB: Model Monitoring with Scikit-Learn
# การติดตามและเฝ้าระวังโมเดล Machine Learning

#%% [markdown]
## 📚 บทนำ (Introduction)
# 
# ### Model Monitoring คืออะไร?
# 
# **Model Monitoring** คือกระบวนการติดตามและเฝ้าระวังประสิทธิภาพของโมเดล Machine Learning 
# หลังจากที่ได้ deploy ไปใช้งานจริง (Production) เพื่อให้มั่นใจว่าโมเดลยังคงทำงานได้อย่างถูกต้อง
# 
# ### ทำไมต้อง Monitor โมเดล?
# 
# 1. **Data Drift** - ข้อมูลในโลกจริงเปลี่ยนแปลงตลอดเวลา
# 2. **Concept Drift** - ความสัมพันธ์ระหว่าง features และ target เปลี่ยนไป
# 3. **Model Degradation** - ประสิทธิภาพโมเดลลดลงเมื่อเวลาผ่านไป
# 4. **Data Quality Issues** - ปัญหาคุณภาพข้อมูล เช่น missing values, outliers
# 
# ### สิ่งที่จะเรียนรู้ใน LAB นี้
# 
# - ✅ Data Quality Monitoring
# - ✅ Model Performance Tracking
# - ✅ Target Drift Detection
# - ✅ Building Monitoring Dashboard

#%% [markdown]
# ---
# ## 🛠️ Section 0: Environment Setup
# ### การติดตั้งและเตรียมสภาพแวดล้อม

#%%
# ติดตั้ง libraries ที่จำเป็น (รันครั้งเดียว)
#